# Phi-4 Mini ONNX Parallel Function Calling Tutorial

Ang notebook na ito ay nagpapakita kung paano gamitin ang Phi-4 Mini kasama ang ONNX Runtime GenAI para sa parallel na pagtawag ng mga function. Ang pagtawag ng function ay nagbibigay-daan sa modelo na matalinong gamitin ang mga external na tools at API batay sa mga kahilingan ng user.

## Pangkalahatang-ideya

Sa tutorial na ito, matututunan mo kung paano:
- I-set up ang Phi-4 Mini gamit ang ONNX Runtime GenAI
- Mag-define ng mga function schema para sa pag-book ng mga flight at hotel
- Gumamit ng guided generation gamit ang Lark grammar para sa structured na output
- Magpatupad ng parallel na pagtawag ng mga function para sa masalimuot na senaryo ng pag-book ng biyahe

## Mga Kinakailangan

Bago patakbuhin ang notebook na ito, tiyakin na mayroon ka:
- Na-download ang Phi-4 Mini ONNX model
- Na-install ang `onnxruntime-genai` package
- Pangunahing kaalaman sa mga konsepto ng pagtawag ng function


## Hakbang 1: I-import ang Mga Kinakailangang Aklatan

Una, i-import natin ang mga kinakailangang aklatan para sa ating pagpapatupad ng pagtawag ng function.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Hakbang 2: Pag-set up at Pag-configure ng Modelo

Ngayon, iko-configure natin ang Phi-4 Mini ONNX model. Siguraduhing palitan ang path ng aktwal na direktoryo ng iyong modelo.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Hakbang 3: I-configure ang Mga Parameter ng Pagbuo

I-set up ang mga parameter ng pagbuo upang kontrolin ang pag-uugali ng output ng modelo. Ang mga setting na ito ay nagbibigay-daan sa tiyak at nakatuong mga tugon para sa pagtawag ng mga function.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Hakbang 4: Tukuyin ang Mga Magagamit na Function

Dito natin tinutukoy ang mga function na maaaring tawagan ng ating AI assistant. Sa halimbawang ito, mayroon tayong dalawang function na may kaugnayan sa paglalakbay:
1. **booking_flight_tickets**: Para sa pag-book ng mga tiket ng eroplano sa pagitan ng mga paliparan
2. **booking_hotels**: Para sa pag-book ng mga tirahan sa hotel

Ang mga depinisyon ng function ay sumusunod sa format ng schema ng pagtawag ng function ng OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Hakbang 5: Mga Pantulong na Function para sa Paggawa ng Gramatika

Ang mga pantulong na function na ito ay nagko-convert ng ating mga depinisyon ng function sa format ng gramatika ng Lark, na ginagamit ng ONNX Runtime GenAI para sa gabay na paggawa. Tinitiyak nito na ang mga output ng modelo ay may wastong mga tawag sa function sa tamang format ng JSON.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Hakbang 6: Subukan ang Pagbuo ng Gramatika

Subukan natin ang mga function para sa pagbuo ng gramatika upang makita kung paano nila ginagawang tamang format ang mga depinisyon ng ating tool.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Hakbang 7: Ihanda ang Prompt ng Sistema at Generator

Ngayon, gagawa tayo ng prompt ng sistema na magpapaliwanag sa modelo tungkol sa mga magagamit na tools at magse-set up ng generator gamit ang mga parameter para sa guided generation.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Hakbang 8: I-initialize ang Generator gamit ang Guided Generation

Ngayon, gagawa tayo ng generator gamit ang mga nakatakdang parameter at ilalapat ang Lark grammar para sa guided generation.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Hakbang 9: Subukan ang Parallel Function Calling

Ngayon, subukan natin ang ating setup gamit ang isang masalimuot na senaryo ng pag-book ng biyahe na nangangailangan ng pagtawag sa maraming function.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Bumuo ng Mga Tawag sa Function

Ang modelo ay ngayon bubuo ng nakaayos na mga tawag sa function batay sa kahilingan ng user. Salamat sa guided generation, ang output ay magiging valid na JSON format na maaaring direktang i-execute.

**Inaasahang Output**: Ang modelo ay dapat bumuo ng mga tawag sa function para sa:
1. `booking_flight_tickets` na may detalye mula Beijing (PEK) patungong Paris (CDG)
2. `booking_hotels` na may detalye ng tirahan sa Paris

Patakbuhin ang cell sa ibaba upang makita ang live na generation:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Konklusyon

🎉 **Binabati kita!** Matagumpay mong naipatupad ang parallel function calling gamit ang Phi-4 Mini sa ONNX Runtime GenAI.

### Mga Natutunan Mo:

1. **Pag-set up ng Model**: Paano i-configure ang Phi-4 Mini gamit ang ONNX Runtime GenAI
2. **Pagde-define ng Function**: Paano magtakda ng mga schema para sa AI function calling
3. **Guided Generation**: Paano gamitin ang Lark grammar para sa structured output generation
4. **Parallel Function Calls**: Paano pamahalaan ang mga kumplikadong request na nangangailangan ng maraming function calls

### Pangunahing Benepisyo:

- ✅ **Structured Output**: Ang guided generation ay nagbibigay ng valid na JSON function calls
- ✅ **Parallel Processing**: Kayang pamahalaan ang maraming function calls sa isang request
- ✅ **High Performance**: Ang ONNX Runtime ay nagbibigay ng optimized na inference
- ✅ **Flexible Schema**: Madaling magdagdag o mag-modify ng mga function definitions

### Mga Mapagkukunan:

- [Phi-4 Mini Documentation](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Documentation](https://onnxruntime.ai/docs/genai/)
- [Function Calling Best Practices](https://platform.openai.com/docs/guides/function-calling)



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
